
<h3 align="center">به نام خدا </h3>
<h2 align="center">تمرین سری پنجم </h2>


<p align="right">در این تمرین داده ها در سه کلاس مختلف و دارای دو ویژگی هستند می خواهیم با کمک دانسته های خود از توزیع های نرمال چند متغیره داده ها را دسته بندی کنیم</p>


<p align="right">دقت کنید که در این تمرین سه مجموعه داده ی آموزشی و یک مجموعه داده آزمایشی داریم. می خواهیم هر چهار حالت مختلفی که برای کوواریانس در جدول ۵٫۱ کتاب آمده است را برای این مجموعه داده پیاده سازی کنیم. یعنی برای هر مجموعه داده ۴ حالت مختلف و در کل ۱۲ حالت مختلف داریم</p>

# داده های ۵۰ تایی

<p align="right">ابتدا داده ها را از فایل می خوانیم</p>

In [0]:
import pandas as pd
import numpy as np

train = pd.read_csv("train_50.csv") 

labels = train['class']
f1 = train['feature_1']
f2 = train['feature2']

In [0]:
A = []
B = []
C = []

for i in range(len(f1)):
  listt = []
  listt.append(f1[i])
  listt.append(f2[i])

  if labels[i] == 'A':
    A.append(listt)
  elif labels[i] == 'B':
    B.append(listt)
  else:
    C.append(listt)
 

In [3]:
cov_A = np.cov(A, rowvar=False)
cov_B = np.cov(B, rowvar=False)
cov_C = np.cov(C, rowvar=False)

print(cov_A, cov_B, cov_C)

mean_A = [np.mean(A, axis=0)]
mean_B = [np.mean(B, axis=0)]
mean_C = [np.mean(C, axis=0)]

print(mean_A, mean_B, mean_C)

pc_A = len(A)/(len(A) + len(B) + len(C))
pc_B = len(B)/(len(A) + len(B) + len(C))
pc_C = len(C)/(len(A) + len(B) + len(C))

print(pc_A, pc_B, pc_C)



[[ 81.78150034 -44.9401266 ]
 [-44.9401266  101.73979937]] [[2.82577979e+01 1.55780796e-02]
 [1.55780796e-02 3.45786294e+01]] [[132.06377012  68.35886693]
 [ 68.35886693  79.74202505]]
[array([-10.21576233,  31.36563462])] [array([35.64136359, -4.92497525])] [array([2.71124905, 1.79400216])]
0.3333333333333333 0.3333333333333333 0.3333333333333333


<p align="right">اکنون با کمک داده های موجود و چهار روشی که در جدول ۵٫۱ آمده  است داده های فایل تست را ارزیابی کنید و نتیجه را گزارش کنید</p>

Case A

In [4]:
import pandas as pd
test = pd.read_csv("test_labels.csv") 

test_labels = test['class']
test_f1 = test['feature_1']
test_f2 = test['feature2']

test_data = []

for i in range(len(test_f1)):

  listt = []
  listt.append(test_f1[i])
  listt.append(test_f2[i])

  test_data.append(listt)


inverse_cov_A = np.linalg.inv(cov_A)
inverse_cov_B = np.linalg.inv(cov_B)
inverse_cov_C = np.linalg.inv(cov_C)


final = []
for x in test_data:
  g = []
  tmp_A = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_A))]
  tmp_B = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_B))]
  tmp_C = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_C))]
  g.append( (-0.5 * np.log(np.linalg.det(cov_A)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_A), inverse_cov_A)  , tmp_A )  )+  np.log(pc_A) )
  g.append( (-0.5 * np.log(np.linalg.det(cov_B)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_B), inverse_cov_B)  , tmp_B )  )+  np.log(pc_B) )
  g.append( (-0.5 * np.log(np.linalg.det(cov_C)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_C), inverse_cov_C)  , tmp_C )  )+  np.log(pc_C) )

  final.append(g)

print(np.shape(final))


(600, 3, 1, 1)


In [5]:
from sklearn.metrics import accuracy_score

preds = []
y_test = []

for x in final:
  preds.append(x.index(np.max(x)))

for label in test_labels:
  if label == 'A':
    y_test.append(0)
  elif label == 'B':
    y_test.append(1)
  else:
    y_test.append(2)
    
print(accuracy_score(y_test, preds))

0.9666666666666667


Case B

In [6]:
s = np.dot(pc_A, cov_A) + np.dot(pc_B, cov_B) + np.dot(pc_C, cov_C)
inverse_s = np.linalg.inv(s)

final_B = []
for x in test_data:
  g = []
  tmp_A = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_A))]
  tmp_B = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_B))]
  tmp_C = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_C))]
  g.append(  ( - 0.5 * np.dot(np.dot(np.transpose(tmp_A), inverse_s)  , tmp_C )  )+  np.log(pc_A) )
  g.append(  ( - 0.5 * np.dot(np.dot(np.transpose(tmp_B), inverse_s)  , tmp_B )  )+  np.log(pc_B) )
  g.append(  ( - 0.5 * np.dot(np.dot(np.transpose(tmp_C), inverse_s)  , tmp_C )  )+  np.log(pc_C) )

  final_B.append(g)

print(np.shape(final_B))

(600, 3, 1, 1)


In [7]:
preds_B = []

for x in final_B:
  preds_B.append(x.index(np.max(x)))

print(accuracy_score(y_test, preds_B))

0.8516666666666667


Case C:

In [8]:
std_A = [np.std(A, axis=0)]
std_B = [np.std(B, axis=0)]
std_C = [np.std(C, axis=0)]

final_C = []

for x in test_data:
  g = []
  tmp_A = np.power(x - np.transpose(mean_A) / std_A, 2)
  tmp_B = np.power(x - np.transpose(mean_B) / std_B, 2)
  tmp_C = np.power(x - np.transpose(mean_C) / std_C, 2)

  
  g.append( -0.5 * sum(sum(tmp_A)) +  np.log(pc_A) )
  g.append( -0.5 * sum(sum(tmp_B)) +  np.log(pc_B) )
  g.append( -0.5 * sum(sum(tmp_C)) +  np.log(pc_C) )

  final_C.append(g)

print(np.shape(final_C))

(600, 3)


In [9]:
preds_C = []

for x in final_C:
  preds_C.append(x.index(np.max(x)))

print(accuracy_score(y_test, preds_C))

0.5033333333333333


Case D

In [10]:

final_D = []

for x in test_data:
  g = []
  g.append( np.dot(mean_A, x) - 0.5 * np.linalg.norm(np.transpose(mean_A)))
  g.append( np.dot(mean_B, x) - 0.5 * np.linalg.norm(np.transpose(mean_B)))
  g.append( np.dot(mean_C, x) - 0.5 * np.linalg.norm(np.transpose(mean_C)))

  final_D.append(g)

print(np.shape(final_D))

(600, 3, 1)


In [11]:
from sklearn.metrics import accuracy_score
preds_D = []

for x in final_D:
  preds_D.append(x.index(np.max(x)))

print(accuracy_score(y_test, preds_D))

0.7166666666666667


# داده های 250 تایی

<p align="right">ابتدا داده ها را از فایل می خوانیم</p>

In [0]:
import pandas as pd
import numpy as np

train = pd.read_csv("train_250.csv") 

labels = train['class']
f1 = train['feature_1']
f2 = train['feature2']

A = []
B = []
C = []

for i in range(len(f1)):
  listt = []
  listt.append(f1[i])
  listt.append(f2[i])

  if labels[i] == 'A':
    A.append(listt)
  elif labels[i] == 'B':
    B.append(listt)
  else:
    C.append(listt)
 

In [4]:
cov_A = np.cov(A, rowvar=False)
cov_B = np.cov(B, rowvar=False)
cov_C = np.cov(C, rowvar=False)

print(cov_A, cov_B, cov_C)

mean_A = [np.mean(A, axis=0)]
mean_B = [np.mean(B, axis=0)]
mean_C = [np.mean(C, axis=0)]

print(mean_A, mean_B, mean_C)

pc_A = len(A)/(len(A) + len(B) + len(C))
pc_B = len(B)/(len(A) + len(B) + len(C))
pc_C = len(C)/(len(A) + len(B) + len(C))

print(pc_A, pc_B, pc_C)


[[105.24418679 -54.52131438]
 [-54.52131438 108.79192294]] [[28.39809024  0.76879621]
 [ 0.76879621 28.31473344]] [[120.23071263  60.82751456]
 [ 60.82751456  75.11387381]]
[array([-10.65756813,  30.72231011])] [array([35.38229744, -4.85565289])] [array([5.04479034, 2.59324312])]
0.3333333333333333 0.3333333333333333 0.3333333333333333


Case A

In [5]:

test = pd.read_csv("test_labels.csv") 

test_labels = test['class']
test_f1 = test['feature_1']
test_f2 = test['feature2']

test_data = []

for i in range(len(test_f1)):

  listt = []
  listt.append(test_f1[i])
  listt.append(test_f2[i])

  test_data.append(listt)


inverse_cov_A = np.linalg.inv(cov_A)
inverse_cov_B = np.linalg.inv(cov_B)
inverse_cov_C = np.linalg.inv(cov_C)


final = []
for x in test_data:
  g = []
  tmp_A = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_A))]
  tmp_B = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_B))]
  tmp_C = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_C))]
  g.append( (-0.5 * np.log(np.linalg.det(cov_A)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_A), inverse_cov_A)  , tmp_A )  )+  np.log(pc_A) )
  g.append( (-0.5 * np.log(np.linalg.det(cov_B)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_B), inverse_cov_B)  , tmp_B )  )+  np.log(pc_B) )
  g.append( (-0.5 * np.log(np.linalg.det(cov_C)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_C), inverse_cov_C)  , tmp_C )  )+  np.log(pc_C) )

  final.append(g)

print(np.shape(final))


(600, 3, 1, 1)


In [6]:
from sklearn.metrics import accuracy_score

preds = []
y_test = []

for x in final:
  preds.append(x.index(np.max(x)))

for label in test_labels:
  if label == 'A':
    y_test.append(0)
  elif label == 'B':
    y_test.append(1)
  else:
    y_test.append(2)
    
print(accuracy_score(y_test, preds))

0.9633333333333334


Case B

In [7]:
s = np.dot(pc_A, cov_A) + np.dot(pc_B, cov_B) + np.dot(pc_C, cov_C)
inverse_s = np.linalg.inv(s)

final_B = []
for x in test_data:
  g = []
  tmp_A = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_A))]
  tmp_B = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_B))]
  tmp_C = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_C))]
  g.append(  ( - 0.5 * np.dot(np.dot(np.transpose(tmp_A), inverse_s)  , tmp_C )  )+  np.log(pc_A) )
  g.append(  ( - 0.5 * np.dot(np.dot(np.transpose(tmp_B), inverse_s)  , tmp_B )  )+  np.log(pc_B) )
  g.append(  ( - 0.5 * np.dot(np.dot(np.transpose(tmp_C), inverse_s)  , tmp_C )  )+  np.log(pc_C) )

  final_B.append(g)

print(np.shape(final_B))

(600, 3, 1, 1)


In [8]:
preds_B = []

for x in final_B:
  preds_B.append(x.index(np.max(x)))

print(accuracy_score(y_test, preds_B))

0.8433333333333334


Case C

In [9]:
std_A = [np.std(A, axis=0)]
std_B = [np.std(B, axis=0)]
std_C = [np.std(C, axis=0)]

final_C = []

for x in test_data:
  g = []
  tmp_A = np.power(x - np.transpose(mean_A) / std_A, 2)
  tmp_B = np.power(x - np.transpose(mean_B) / std_B, 2)
  tmp_C = np.power(x - np.transpose(mean_C) / std_C, 2)

  
  g.append( -0.5 * sum(sum(tmp_A)) +  np.log(pc_A) )
  g.append( -0.5 * sum(sum(tmp_B)) +  np.log(pc_B) )
  g.append( -0.5 * sum(sum(tmp_C)) +  np.log(pc_C) )

  final_C.append(g)

print(np.shape(final_C))

(600, 3)


In [10]:
preds_C = []

for x in final_C:
  preds_C.append(x.index(np.max(x)))

print(accuracy_score(y_test, preds_C))

0.5133333333333333


Case D

In [11]:
final_D = []

for x in test_data:
  g = []
  
  g.append( np.dot(mean_A, x) - 0.5 * np.linalg.norm(np.transpose(mean_A)))
  g.append( np.dot(mean_B, x) - 0.5 * np.linalg.norm(np.transpose(mean_B)))
  g.append( np.dot(mean_C, x) - 0.5 * np.linalg.norm(np.transpose(mean_C)))

  final_D.append(g)

print(np.shape(final_D))


(600, 3, 1)


In [12]:
preds_D = []

for x in final_D:
  preds_D.append(x.index(np.max(x)))

print(accuracy_score(y_test, preds_D))

0.7133333333333334


<p align="right">اکنون با کمک داده های موجود و چهار روشی که در جدول ۵٫۱ آمده  است داده های فایل تست را ارزیابی کنید و نتیجه را گزارش کنید</p>

# داده های 800 تایی

In [0]:
import pandas as pd
import numpy as np

train = pd.read_csv("train_800.csv") 

labels = train['class']
f1 = train['feature_1']
f2 = train['feature2']

A = []
B = []
C = []

for i in range(len(f1)):
  listt = []
  listt.append(f1[i])
  listt.append(f2[i])

  if labels[i] == 'A':
    A.append(listt)
  elif labels[i] == 'B':
    B.append(listt)
  else:
    C.append(listt)
 

In [2]:
cov_A = np.cov(A, rowvar=False)
cov_B = np.cov(B, rowvar=False)
cov_C = np.cov(C, rowvar=False)

print(cov_A, cov_B, cov_C)

mean_A = [np.mean(A, axis=0)]
mean_B = [np.mean(B, axis=0)]
mean_C = [np.mean(C, axis=0)]

print(mean_A, mean_B, mean_C)

pc_A = len(A)/(len(A) + len(B) + len(C))
pc_B = len(B)/(len(A) + len(B) + len(C))
pc_C = len(C)/(len(A) + len(B) + len(C))

print(pc_A, pc_B, pc_C)


[[109.64763501 -51.25302335]
 [-51.25302335  93.45032984]] [[28.9142815  -0.56613003]
 [-0.56613003 26.2125475 ]] [[114.17787964  57.65006723]
 [ 57.65006723  74.59239028]]
[array([-10.12861068,  29.88720546])] [array([35.28391833, -4.96562922])] [array([5.16869842, 2.02723897])]
0.3333333333333333 0.3333333333333333 0.3333333333333333


Case A

In [3]:
test = pd.read_csv("test_labels.csv") 

test_labels = test['class']
test_f1 = test['feature_1']
test_f2 = test['feature2']

test_data = []

for i in range(len(test_f1)):

  listt = []
  listt.append(test_f1[i])
  listt.append(test_f2[i])

  test_data.append(listt)


inverse_cov_A = np.linalg.inv(cov_A)
inverse_cov_B = np.linalg.inv(cov_B)
inverse_cov_C = np.linalg.inv(cov_C)


final = []
for x in test_data:
  g = []
  tmp_A = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_A))]
  tmp_B = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_B))]
  tmp_C = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_C))]
  g.append( (-0.5 * np.log(np.linalg.det(cov_A)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_A), inverse_cov_A)  , tmp_A )  )+  np.log(pc_A) )
  g.append( (-0.5 * np.log(np.linalg.det(cov_B)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_B), inverse_cov_B)  , tmp_B )  )+  np.log(pc_B) )
  g.append( (-0.5 * np.log(np.linalg.det(cov_C)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_C), inverse_cov_C)  , tmp_C )  )+  np.log(pc_C) )

  final.append(g)

print(np.shape(final))

(600, 3, 1, 1)


In [4]:
from sklearn.metrics import accuracy_score

preds = []
y_test = []

for x in final:
  preds.append(x.index(np.max(x)))

for label in test_labels:
  if label == 'A':
    y_test.append(0)
  elif label == 'B':
    y_test.append(1)
  else:
    y_test.append(2)
    
print(accuracy_score(y_test, preds))

0.9633333333333334


Case B

In [5]:
s = np.dot(pc_A, cov_A) + np.dot(pc_B, cov_B) + np.dot(pc_C, cov_C)
inverse_s = np.linalg.inv(s)

final_B = []
for x in test_data:
  g = []
  tmp_A = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_A))]
  tmp_B = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_B))]
  tmp_C = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_C))]
  g.append(  ( - 0.5 * np.dot(np.dot(np.transpose(tmp_A), inverse_s)  , tmp_C )  )+  np.log(pc_A) )
  g.append(  ( - 0.5 * np.dot(np.dot(np.transpose(tmp_B), inverse_s)  , tmp_B )  )+  np.log(pc_B) )
  g.append(  ( - 0.5 * np.dot(np.dot(np.transpose(tmp_C), inverse_s)  , tmp_C )  )+  np.log(pc_C) )

  final_B.append(g)

print(np.shape(final_B))

(600, 3, 1, 1)


In [6]:
preds_B = []

for x in final_B:
  preds_B.append(x.index(np.max(x)))

print(accuracy_score(y_test, preds_B))

0.8383333333333334


Case C

In [7]:
std_A = [np.std(A, axis=0)]
std_B = [np.std(B, axis=0)]
std_C = [np.std(C, axis=0)]

final_C = []

for x in test_data:
  g = []
  tmp_A = np.power(x - np.transpose(mean_A) / std_A, 2)
  tmp_B = np.power(x - np.transpose(mean_B) / std_B, 2)
  tmp_C = np.power(x - np.transpose(mean_C) / std_C, 2)

  
  g.append( -0.5 * sum(sum(tmp_A)) +  np.log(pc_A) )
  g.append( -0.5 * sum(sum(tmp_B)) +  np.log(pc_B) )
  g.append( -0.5 * sum(sum(tmp_C)) +  np.log(pc_C) )

  final_C.append(g)

print(np.shape(final_C))

(600, 3)


In [8]:
preds_C = []

for x in final_C:
  preds_C.append(x.index(np.max(x)))

print(accuracy_score(y_test, preds_C))

0.5216666666666666


Case D

In [9]:
final_D = []

for x in test_data:
  g = []
  
  g.append( np.dot(mean_A, x) - 0.5 * np.linalg.norm(np.transpose(mean_A), 2))
  g.append( np.dot(mean_B, x) - 0.5 * np.linalg.norm(np.transpose(mean_B), 2))
  g.append( np.dot(mean_C, x) - 0.5 * np.linalg.norm(np.transpose(mean_C), 2))

  final_D.append(g)

print(np.shape(final_D))

(600, 3, 1)


In [10]:
preds_D = []

for x in final_D:
  preds_D.append(x.index(np.max(x)))

print(accuracy_score(y_test, preds_D))

0.7133333333333334


# ارزیابی بهترین مدل

<p align="right">در این بخش داده های موجود را با بهترین مدلی که یافتید دسته بندی کنید و نتایج را در یک فایل ذخیره کنید. در این فایل فقط کلاس داده ها را به ترتیب فایل داده شده بنویسید. دقت این ارزیابی  توسط دستیار آموزشی محاسبه می گردد و در نمره شما موثر است</p>

<p align="right">ابتدا داده ها را از فایل می خوانیم</p>

In [0]:
import pandas as pd
test = pd.read_csv("test_without_labels.csv") 

In [12]:
f1 = test['feature_1']
f2 = test['feature2']

fi = []

data = []

for i in range(len(f1)):

  l = []
  l.append(f1[i])
  l.append(f2[i])

  data.append(l)

for x in data:
  g = []
  tmp_A = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_A))]
  tmp_B = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_B))]
  tmp_C = [a_i - b_i for a_i, b_i in zip(x, np.transpose(mean_C))]
  g.append( (-0.5 * np.log(np.linalg.det(cov_A)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_A), inverse_cov_A)  , tmp_A )  )+  np.log(pc_A) )
  g.append( (-0.5 * np.log(np.linalg.det(cov_B)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_B), inverse_cov_B)  , tmp_B )  )+  np.log(pc_B) )
  g.append( (-0.5 * np.log(np.linalg.det(cov_C)) ) - (  0.5 * np.dot(np.dot(np.transpose(tmp_C), inverse_cov_C)  , tmp_C )  )+  np.log(pc_C) )

  fi.append(g)

print(np.shape(fi))

(1500, 3, 1, 1)


In [0]:
p = []
y_test = []

for x in fi:
  p.append(x.index(np.max(x)))

In [0]:
f = open("predictions.txt", "w")
for listitem in p:
  if listitem == 0:
    f.write("A")
  elif listitem == 1:
    f.write("B")
  else:
    f.write("C")
  f.write("\n")  

f.close()